## Data Collection


We will be using data from here: https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes

In [1]:
import numpy as np
import pandas as pd
from os import listdir
from functools import reduce

In [2]:
#data is stored in multiple sheets, separated by make
cars = dict()
for sheet in listdir('Data'):
    make = sheet.split('.')[0]
    readSheet = pd.read_csv('Data\\' + sheet)
    readSheet.insert(0, 'make', make)
    cars[make] = readSheet
sheetColumns = [c.columns.to_numpy().tolist() for c in cars.values()]
for s in sheetColumns:
    print(s)

['make', 'model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']
['make', 'model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']
['make', 'model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']
['make', 'model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax(£)', 'mpg', 'engineSize']
['make', 'model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']
['make', 'model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']
['make', 'model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']
['make', 'model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']
['make', 'model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']


In [3]:
#hyundai has "(£)" which needs to be removed before concat
cars['hyundai'].rename(columns={'tax(£)':'tax'}, inplace=True)
cars['hyundai'].head()

,make,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,hyundai,I20,2017,7999,Manual,17307,Petrol,145,58.9,1.2
1,hyundai,Tucson,2016,14499,Automatic,25233,Diesel,235,43.5,2.0
2,hyundai,Tucson,2016,11399,Manual,37877,Diesel,30,61.7,1.7
3,hyundai,I10,2016,6499,Manual,23789,Petrol,20,60.1,1.0
4,hyundai,IX35,2015,10199,Manual,33177,Diesel,160,51.4,2.0


In [4]:
#now that we know what the columns are we can combine the data
if type(cars) != type(pd.DataFrame()): cars = pd.concat(cars.values())
cars.shape

(99187, 10)

In [5]:
cars.describe()

,year,price,mileage,tax,mpg,engineSize
count,99187.000000,99187.000000,99187.000000,99187.000000,99187.000000,99187.000000
mean,2017.087723,16805.347656,23058.914213,120.299838,55.166825,1.663280
std,2.123934,9866.773417,21148.523721,63.150926,16.138522,0.557646
min,1970.000000,450.000000,1.000000,0.000000,0.300000,0.000000
25%,2016.000000,9999.000000,7425.000000,125.000000,47.100000,1.200000
50%,2017.000000,14495.000000,17460.000000,145.000000,54.300000,1.600000
75%,2019.000000,20870.000000,32339.000000,145.000000,62.800000,2.000000
max,2060.000000,159999.000000,323000.000000,580.000000,470.800000,6.600000


make: maker of car

model: model of car

year: year of manufacture

price: sold price

transmission: type of transmission

mileage: miles at time of sale

fuelType: fuel type

tax: tax paid on purchase

mpg: fuel econ. in miles per gallon

engineSize: Engine volume in liters

In [6]:
#check for missing values
cars.isna().sum()

make            0
model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

In [7]:
cars.dtypes

make             object
model            object
year              int64
price             int64
transmission     object
mileage           int64
fuelType         object
tax               int64
mpg             float64
engineSize      float64
dtype: object

In [8]:
#check for object columns that could be categories
cars.nunique()

make                9
model             195
year               27
price           13236
transmission        4
mileage         42214
fuelType            5
tax                48
mpg               208
engineSize         40
dtype: int64

In [9]:
cars['make'] = cars['make'].astype('category')
cars['model'] = cars['model'].astype('category')
cars['transmission'] = cars['transmission'].astype('category')
cars['fuelType'] = cars['fuelType'].astype('category')
cars.dtypes

make            category
model           category
year               int64
price              int64
transmission    category
mileage            int64
fuelType        category
tax                int64
mpg              float64
engineSize       float64
dtype: object

In [10]:
#check for duplicate records
cars.duplicated().sum()

1475

In [11]:
#what are they
cars[cars.duplicated(keep=False)].sort_values(by='price')

,make,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
15684,ford,KA,2008,1795,Manual,63000,Petrol,160,45.6,1.3
15685,ford,KA,2008,1795,Manual,63000,Petrol,160,45.6,1.3
12444,vw,Up,2014,4330,Manual,49160,Petrol,20,62.8,1.0
12502,vw,Up,2014,4330,Manual,49160,Petrol,20,62.8,1.0
4379,hyundai,I30,2015,4492,Manual,124000,Diesel,0,78.4,1.6
...,...,...,...,...,...,...,...,...,...,...
2956,merc,S Class,2020,64980,Semi-Auto,3999,Diesel,145,40.4,3.0
4999,audi,RS5,2019,66990,Automatic,6000,Petrol,145,29.7,2.9
3997,audi,RS5,2019,66990,Automatic,6000,Petrol,145,29.7,2.9
5238,merc,GLC Class,2019,74980,Semi-Auto,1998,Petrol,145,21.6,4.0


In [12]:
#duplicates seem to be true exact duplicates, drop them.
cars = cars.drop_duplicates()
cars.duplicated().sum()

0

In [13]:
#recheck for outliers
cars.describe()

,year,price,mileage,tax,mpg,engineSize
count,97712.000000,97712.000000,97712.000000,97712.000000,97712.000000,97712.000000
mean,2017.066870,16773.487555,23219.475499,120.142408,55.205623,1.664913
std,2.122993,9868.552222,21060.882301,63.357250,16.181659,0.558574
min,1970.000000,450.000000,1.000000,0.000000,0.300000,0.000000
25%,2016.000000,9999.000000,7673.000000,125.000000,47.100000,1.200000
50%,2017.000000,14470.000000,17682.500000,145.000000,54.300000,1.600000
75%,2019.000000,20750.000000,32500.000000,145.000000,62.800000,2.000000
max,2060.000000,159999.000000,323000.000000,580.000000,470.800000,6.600000


In [14]:
#year max should not be 2060
cars[cars.year>2020]

,make,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
17726,ford,Fiesta,2060,6495,Automatic,54807,Petrol,205,42.8,1.4


In [15]:
#we can afford to lose one record
cars = cars[cars.year<=2020]
#min mpg seems questionable
cars[cars.mpg<10]

,make,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
1237,bmw,X3,2020,52910,Semi-Auto,101,Hybrid,135,5.5,2.0
2116,bmw,X3,2020,54990,Semi-Auto,2137,Hybrid,135,5.5,2.0
3138,bmw,3 Series,2019,35230,Semi-Auto,4602,Hybrid,135,8.8,2.0
4991,bmw,3 Series,2019,33999,Semi-Auto,8680,Hybrid,135,8.8,2.0
5515,bmw,3 Series,2019,33995,Semi-Auto,7974,Hybrid,135,8.8,2.0
5960,bmw,3 Series,2019,33699,Semi-Auto,7736,Hybrid,135,8.8,2.0
6132,bmw,X3,2020,47000,Semi-Auto,1385,Hybrid,135,5.5,2.0
6172,bmw,X3,2020,42990,Semi-Auto,3245,Hybrid,140,5.5,2.0
6198,bmw,X3,2020,53995,Semi-Auto,2000,Hybrid,135,5.5,2.0
6489,bmw,3 Series,2019,35995,Semi-Auto,2166,Hybrid,135,8.8,2.0


In [16]:
#something about the mpg calculation is clearly wrong or at least measured in an inaccurate way
#also a max of 470 seems pretty extreme
#there's clearly some calculation being made here
#the most the manufacturer's ever advertise is 50-55, the highest I found online in a quick search was 60
cars[cars.mpg>60]

,make,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
1,audi,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
3,audi,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
6,audi,A6,2016,13250,Automatic,76788,Diesel,30,61.4,2.0
7,audi,A4,2016,11750,Manual,75185,Diesel,20,70.6,2.0
8,audi,A3,2015,10200,Manual,46112,Petrol,20,60.1,1.4
...,...,...,...,...,...,...,...,...,...,...
15106,vw,CC,2016,13000,Manual,19000,Diesel,30,62.8,2.0
15110,vw,CC,2012,6695,Manual,98000,Diesel,30,60.1,2.0
15132,vw,Caddy Life,2017,14990,Manual,15986,Diesel,145,60.1,2.0
15136,vw,Caddy,2015,12490,Manual,54641,Diesel,265,60.1,2.0


In [18]:
#that's far too many records to just drop or write off as incorrect
#remember this when looking at mpg later

#NOTE: After completing this part of the task, I learned that the UK uses a larger gallon.
#This accounts for the higher mpg. Most of the extremely high mpg records are hybrids
#or other atypical cars.

#last check, no way engine size can be zero
#quick web search shows that .6 L is a reasonable number for the very smallest an engine could/would/should be
cars[cars.engineSize<.6]

,make,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
7505,audi,Q5,2019,44790,Automatic,5886,Petrol,135,117.7,0.0
7506,audi,Q3,2019,32788,Automatic,1500,Diesel,145,47.1,0.0
7516,audi,Q3,2020,29944,Manual,1500,Petrol,145,40.9,0.0
7517,audi,Q3,2020,33333,Automatic,1500,Diesel,145,47.1,0.0
7518,audi,Q3,2020,29944,Automatic,1500,Petrol,145,32.5,0.0
...,...,...,...,...,...,...,...,...,...,...
11648,vw,Tiguan,2016,15300,Manual,38398,Diesel,145,53.3,0.0
11655,vw,Tiguan,2018,24000,Automatic,22200,Diesel,145,47.9,0.0
12821,vw,Up,2017,8500,Manual,20324,Petrol,20,64.2,0.0
12879,vw,Up,2017,8000,Manual,24444,Petrol,20,60.1,0.0


In [19]:
#check for any non-zero
cars[(cars.engineSize<.6) & (cars.engineSize!=0)]

,make,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize


In [20]:
#unfortunately, cars can be customized so once again we can't assume anything to fill in these records
cars=cars[cars.engineSize!=0]

In [21]:
#remove excess whitespace from categories
cars.loc[:,'make'] = cars.make.str.strip()
cars.loc[:,'model'] = cars.model.str.strip()
cars.loc[:,'transmission'] = cars.transmission.str.strip()
cars.loc[:,'fuelType'] = cars.fuelType.str.strip()

In [22]:
#save organized/cleaned data
cars.reset_index(inplace=True, drop=True)
cars.to_csv('completeData.csv', index=False)

In [23]:
pd.read_csv('completeData.csv').head()

,make,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,audi,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,audi,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,audi,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,audi,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,audi,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0
